In [1]:
from glob import glob
import sets
import datetime
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np
from collections import defaultdict

/Users/along528/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: the sets module is deprecated
  from ipykernel import kernelapp as app


## Load data frames

In [8]:
dbname_police = 'census'
username = 'along528'
pswd = 'password'
con_census = None
con_census = psycopg2.connect(database = dbname_police, user = username, host='localhost', password=pswd)

In [45]:
census_df = pd.read_sql('SELECT * FROM census_by_zipcode',con_census).drop('index',axis=1)
census_df['zipcode'] = pd.to_numeric(census_df['zipcode'])
census_df = census_df.drop(['median_income_estimate_all', 'median_income_error_all', 
 'median_income_estimate_white', 'median_income_error_white', 
 'median_income_estimate_black', 'median_income_error_black'],axis=1)
census_df = census_df.set_index('zipcode')
census_df

,total,urban,rural,institutionalized_all,institutionalized_adult_all,institutionalized_adult_federal_detention_all,institutionalized_adult_federal_prison_all,institutionalized_adult_state_prison_all,institutionalized_adult_local_jail_all,institutionalized_juvenile_all,...,institutionalized_adult_state_prison_black,institutionalized_adult_local_jail_black,institutionalized_juvenile_black,population_white,population_black,population_native_american,population_asian,population_nathaw,population_other_race,population_mult_race
zipcode,,,,,,,,,,,,,,,,,,,,,
601,18570,10679,7891,57,0,0,0,0,0,20,...,0,0,0,17285,572,65,5,1,402,240
602,41520,41520,0,12,0,0,0,0,0,0,...,0,0,0,35980,2210,129,22,0,2181,998
603,54689,54646,43,826,727,0,0,727,0,17,...,87,0,0,45348,4141,140,85,4,3614,1357
606,6615,2697,3918,0,0,0,0,0,0,0,...,0,0,0,5883,314,27,3,8,280,100
610,29016,25640,3376,62,0,0,0,0,0,0,...,0,0,0,23796,2083,100,37,4,2175,821
612,67010,62391,4619,185,69,0,0,69,0,12,...,6,0,0,56488,4130,241,79,6,3964,2102
616,11017,10438,579,34,0,0,0,0,0,0,...,0,0,0,9339,786,38,4,0,670,180
617,24597,24161,436,28,0,0,0,0,0,0,...,0,0,0,19831,1885,81,30,0,2159,611
622,7853,6171,1682,46,0,0,0,0,0,0,...,0,0,0,6738,393,13,3,2,483,221


In [67]:
census_all_df = pd.read_sql('SELECT * FROM census_by_zipcode',con_census).drop('index',axis=1)
census_all_df['zipcode'] = pd.to_numeric(census_all_df['zipcode'])
census_all_df = census_all_df.set_index('zipcode')
census_all_df

,total,urban,rural,institutionalized_all,institutionalized_adult_all,institutionalized_adult_federal_detention_all,institutionalized_adult_federal_prison_all,institutionalized_adult_state_prison_all,institutionalized_adult_local_jail_all,institutionalized_juvenile_all,...,population_asian,population_nathaw,population_other_race,population_mult_race,median_income_estimate_all,median_income_error_all,median_income_estimate_white,median_income_error_white,median_income_estimate_black,median_income_error_black
zipcode,,,,,,,,,,,,,,,,,,,,,
601,18570,10679,7891,57,0,0,0,0,0,20,...,5,1,402,240,10833,1531,10848,1537,35294,18228
602,41520,41520,0,12,0,0,0,0,0,0,...,22,0,2181,998,16353,977,15906,1033,17371,5466
603,54689,54646,43,826,727,0,0,727,0,17,...,85,4,3614,1357,16323,872,16006,897,16346,9041
606,6615,2697,3918,0,0,0,0,0,0,0,...,3,8,280,100,14138,2157,14773,2261,9412,6540
610,29016,25640,3376,62,0,0,0,0,0,0,...,37,4,2175,821,17265,1065,16813,1221,13684,1611
612,67010,62391,4619,185,69,0,0,69,0,12,...,79,6,3964,2102,17752,783,17873,849,19003,4045
616,11017,10438,579,34,0,0,0,0,0,0,...,4,0,670,180,15115,2164,15089,2270,34286,39868
617,24597,24161,436,28,0,0,0,0,0,0,...,30,0,2159,611,15461,1514,15621,1573,17297,3377
622,7853,6171,1682,46,0,0,0,0,0,0,...,3,2,483,221,14993,3916,14781,4186,0,0


In [7]:
dbname_police = 'police_bjs'
username = 'along528'
pswd = 'password'
con_survey = None
con_survey = psycopg2.connect(database = dbname_police, user = username, host='localhost', password=pswd)

In [13]:
survey_df = pd.read_sql('SELECT * FROM police_data_table',con_survey)
survey_df

,SURVEYID,formtype,resptype,agcytype,agency,city,state,zipcode,swnauthemp,swnftemp,...,impgangtask,imphumntask,impterrtask,impcell,ori,csllea04_id,population,lpdsampgrp,finalwt_page1,finalwt_page2on
0,1,L,full,5,ALASKA STATE TROOOPERS,Anchorage,AK,99811,388,366,...,0,0,0,,AKAST01,10045062,,,1.088889,1.088889
1,3,L,full,5,ALABAMA DEPARTMENT OF PUBLIC,Montgomery,AL,36102,668,668,...,0,0,0,,ALAST00,10045061,,,1.088889,1.088889
2,17,L,full,5,ARKANSAS STATE POLICE,Little Rock,AR,72209,580,536,...,0,0,0,,ARASP00,10045064,,,1.088889,1.088889
3,26,L,full,5,ARIZONA DEPARTMENT OF PUBLIC,Phoenix,AZ,85005,1398,1257,...,0,0,0,,AZCCHPX,10045063,,,1.088889,1.088889
4,53,L,full,5,CALIFORNIA HIGHWAY PATROL,Sacramento,CA,94298,7507,7146,...,0,0,0,,CA03499,10045065,,,1.088889,1.088889
5,153,L,full,5,COLORADO STATE PATROL,Lakewood,CO,80215,717,717,...,0,0,0,,COCSP00,10045066,,,1.088889,1.088889
6,169,L,full,5,CONNECTICUT STATE POLICE,Middletown,CT,6457,1255,1255,...,0,0,0,,CTCSP00,10045067,,,1.088889,1.088889
7,188,L,full,5,DELAWARE STATE POLICE,Dover,DE,19903,665,677,...,0,0,0,,DEDSP00,10045068,,,1.088889,1.088889
8,213,L,full,5,FLORIDA HIGHWAY PATROL,Tallahassee,FL,32399,1669,1664,...,0,0,0,,FLFHP00,10045069,,,1.088889,1.088889
9,305,L,full,5,GEORGIA DEPT OF PUBLIC SAFETY,Atlanta,GA,30371,1232,1030,...,0,0,0,,GAGSP00,10045070,,,1.088889,1.088889


In [5]:
dbname = 'combined_profiling'
username = 'along528'
pswd = 'password'
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))

In [3]:
dbname = 'combined_profiling'
username = 'along528'
pswd = 'password'
con_comb = None
con_comb = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [37]:
zip_df = pd.read_sql('SELECT * FROM zipcode_map',con_comb).drop('index',axis=1)
zip_df['Postal Code'] = zip_df['Postal Code'].astype(int)

## Sum census population data based on zip code

In [66]:
count=0
matched_series = []
for survey_index,survey_series in survey_df.iterrows():
    surveyid = survey_series['SURVEYID']
    #print surveyid
    zip_matched_df = zip_df[zip_df['surveyid_city']==surveyid]
    if len(zip_matched_df)==0: continue
    count+=1
    zip_matched_list = zip_matched_df['Postal Code'].tolist()
    census_matched_series = census_df[census_df.index.isin(zip_matched_list)].sum() #, 'b'].sum()
    census_matched_series['surveyid'] = surveyid
    matched_series.append(census_matched_series)
    #if count > 10: break
#zip_matched_df
census_matched_sum_df = pd.DataFrame(pd.concat(matched_series,axis=1)).transpose()
census_matched_sum_df

,total,urban,rural,institutionalized_all,institutionalized_adult_all,institutionalized_adult_federal_detention_all,institutionalized_adult_federal_prison_all,institutionalized_adult_state_prison_all,institutionalized_adult_local_jail_all,institutionalized_juvenile_all,...,institutionalized_adult_local_jail_black,institutionalized_juvenile_black,population_white,population_black,population_native_american,population_asian,population_nathaw,population_other_race,population_mult_race,surveyid
0,242037,238222,3815,2392,1001,0,0,0,629,190,...,70,11,151441,14039,21549,22442,5724,6154,20688,2
1,13508,10352,3156,120,69,0,0,69,0,30,...,0,3,9204,79,1912,943,27,93,1250,1001
2,4005,0,4005,17,2,0,0,0,2,0,...,0,0,3270,24,326,76,32,26,251,1003
3,144195,144195,0,1960,1115,0,0,910,0,36,...,0,11,57059,49829,789,4917,151,25194,6256,167
4,112654,106927,5727,942,615,0,0,0,615,68,...,366,29,62416,45082,262,1883,23,1767,1221,16
5,270712,262620,8092,2785,1320,0,0,0,1320,131,...,856,90,136887,121541,948,4901,141,2453,3841,13
6,70818,58972,11846,1055,467,0,0,0,467,77,...,194,27,48199,18934,282,618,28,1618,1139,1004
7,493806,487923,5883,4197,1391,0,0,280,1111,258,...,747,178,248229,216662,1109,10220,194,11592,5800,5
8,214703,202917,11786,5170,3752,325,914,2004,509,305,...,406,200,81206,120864,535,4643,166,4545,2744,15
9,80446,66438,14008,1171,553,0,0,0,439,46,...,251,25,53580,23271,297,764,45,1071,1418,7


In [68]:
census_matched_sum_df.to_sql('census_population_sum',engine,if_exists='replace')